<a href="https://colab.research.google.com/github/dcyphr/dcyphr-NLP/blob/master/PERSONAL_PEGASUS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SETUP AND INSTALLATION

In [ ]:
!nvidia-smi

Sat Aug 29 22:30:15 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%bash
cd /content/
git clone https://github.com/dcyphr/dcyphr-NLP.git


fatal: destination path 'dcyphr-NLP' already exists and is not an empty directory.


In [ ]:
!git clone https://github.com/huggingface/transformers.git
%cd transformers
!pip install -q -e .
%cd examples
!pip install -U -q -r requirements.txt
!pip install -U pyarrow

Cloning into 'transformers'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 39749 (delta 5), reused 7 (delta 2), pack-reused 39728
Receiving objects: 100% (39749/39749), 28.57 MiB | 28.05 MiB/s, done.
Resolving deltas: 100% (27591/27591), done.
/content/transformers/examples/seq2seq/transformers/examples/transformers
/content/transformers/examples/seq2seq/transformers/examples/transformers/examples
Requirement already up-to-date: pyarrow in /usr/local/lib/python3.6/dist-packages (1.0.1)


In [ ]:
%cd /content/transformers/examples/seq2seq/

/content/transformers/examples/seq2seq


This installs the XSUM Dataset and is unneccesary if you are working with your own dataset

In [ ]:
!wget https://s3.amazonaws.com/datasets.huggingface.co/summarization/xsum.tar.gz
!tar -xzvf xsum.tar.gz
!export XSUM_DIR=${PWD}/xsum

--2020-08-29 22:30:39--  https://s3.amazonaws.com/datasets.huggingface.co/summarization/xsum.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.30.38
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.30.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 204844092 (195M) [application/x-tar]
Saving to: ‘xsum.tar.gz.1’

xsum.tar.gz.1       100%[===================>] 195.35M  48.7MB/s    in 4.4s    

2020-08-29 22:30:43 (44.3 MB/s) - ‘xsum.tar.gz.1’ saved [204844092/204844092]

xsum/
xsum/train.target
xsum/train.source
xsum/val.source
xsum/val.target
xsum/test.source
xsum/test.target


This Finetunes the PEGASUS Model for One epoch on the Dcyphr fine-tuning data

In [ ]:
!bash finetune_pegasus_xsum.sh \
  --model_name_or_path google/pegasus-xsum \
  --data_dir /content/dcyphr-NLP/fine_tuning_data \
  --output_dir xsum_pegasus_test_5 \
  --train_batch_size 2 \
  --eval_batch_size 2 \
  --num_train_epochs 1 \
  --n_train 100 \
  --n_val 100 \
  --sortish_sampler \
  --gpus 1 \
  --val_check_interval 0.25 \
  --gradient_accumulation_steps 4 \

PyTorch version 1.6.0+cu101 available.
2020-08-29 22:30:51.548899: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
TensorFlow version 2.3.0 available.
Traceback (most recent call last):
  File "finetune.py", line 394, in <module>
    main(args)
  File "finetune.py", line 332, in main
    raise ValueError("Output directory ({}) already exists and is not empty.".format(args.output_dir))
ValueError: Output directory (xsum_pegasus_test_5) already exists and is not empty.


In [ ]:
%%bash
mkdir gens
export DATA_DIR=xsum
python run_eval.py google/pegasus-xsum \
    $DATA_DIR/test.source gens/peg_xsum_test_generation.txt \
    --reference_path $DATA_DIR/test.target \
    --score_path gens/peg_xsum_rouge.txt --task summarization \
    --device cuda \
    --bs 8

**Training on Dcyphr Data**

Command to Finetune on the dcyphr dataset

In [ ]:
%%bash
export DATA_DIR=/content/dcyphr-NLP/fine_tuning_data
python run_eval.py sshleifer/distilbart-cnn-12-6 $DATA_DIR/val.source dbart_val_generations.txt \
    --reference_path $DATA_DIR/val.target \
    --score_path cnn_rouge.json \
    --task summarization \
    --n_obs 100 \
    --device cuda \
    --bs 32

In [ ]:
%%bash
ls

Okay so the output of the data can be found at transformers/examples/seq2seq/dbart_val_generation

In [ ]:
output_dir=output_dir= 'transformers/examples/seq2seq/xsum_pegasus_test_5'
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained(f'{output_dir}/best_tfmr')

PyTorch version 1.6.0+cu101 available.
TensorFlow version 2.3.0 available.
loading configuration file transformers/examples/seq2seq/xsum_pegasus_test_5/best_tfmr/config.json
Model config PegasusConfig {
  "activation_dropout": 0.1,
  "activation_function": "relu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "PegasusForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 16,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 16,
  "eos_token_id": 1,
  "extra_pos_embeddings": 1,
  "force_bos_token_to_be_generated": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,

In [ ]:
 from transformers import BartTokenizer, BartModel

In [ ]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

ARTICLE_TO_SUMMARIZE = "My friends are cool but they eat too many carbs."
inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors='pt')

# Generate Summary
summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=5, early_stopping=True)
print([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids])

https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-vocab.json not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmp4lgyibjs
storing https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-vocab.json in cache at /root/.cache/torch/transformers/1ae1f5b6e2b22b25ccc04c000bb79ca847aa226d0761536b011cf7e5868f0655.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b
creating metadata file for /root/.cache/torch/transformers/1ae1f5b6e2b22b25ccc04c000bb79ca847aa226d0761536b011cf7e5868f0655.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b
https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-merges.txt not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmp7dboqvju
storing https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-merges.txt in cache at /root/.cache/torch/transformers/f8f83199a6270d582d6245dc100e99c4155de81c9745c624807

[' IP sides statute']
